In [ ]:
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import Library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import datetime as dt
from sklearn.cluster import KMeans
%matplotlib inline

plt.rc('font', family='NanumBarunGothic')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
scr = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SC/CARD_SPENDING_RESIDENT.txt', sep='\t', encoding='cp949')
ska = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SK/ska.csv', index_col='Unnamed: 0')

dong = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/지역데이터/전체동별클러스터링.csv', sep = ',', encoding='utf-8')
dong_cd = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SC/SC동코드변환.csv', sep = ',', encoding='utf-8')

In [ ]:
# 유동인구 전처리
ska['STD_YMD'] = pd.to_datetime(ska['STD_YMD'].astype(str), format='%Y%m%d',infer_datetime_format=True)

ska['유동인구_노원구학원가'] = pd.concat([ska.iloc[:,6:8], ska.iloc[:,12:16], ska.iloc[:,21:23], ska.iloc[:,27:31]], axis=1).sum(axis=1) # 10,40,50
ska['유동인구_노원구대학가'] = pd.concat([ska.iloc[:,6:10], ska.iloc[:,12:16], ska.iloc[:,21:25], ska.iloc[:,27:31]], axis=1).sum(axis=1) # 10,20,40,50
ska['유동인구_중구소상공'] =pd.concat([ska.iloc[:,8:12], ska.iloc[:,23:27]], axis=1).sum(axis=1) # 20,
ska['유동인구_중구빌딩숲'] = pd.concat([ska.iloc[:,12:19], ska.iloc[:,27:34]], axis=1).sum(axis=1) # 40,50

In [ ]:
# 신한카드 전처리

# 코드 -> 이름으로 바꾸는 함수 : name_a_to_b
def name_a_to_b(data,col,a,b):
  data = data.replace({col:a}, {col:b}, inplace=True)
  return(data)

# 카테고리 코드 -> 이름
cat_code = [10,20,21,22,30,31,32,33,34,35,40,42,43,44,50,52,60,62,70,71,80,81,92]
cat_name = ['숙박','레저용품','레저업소','문화취미','가구','전기','주방용구',
            '연료판매','광학제품','가전','유통업','의복','직물','신변잡화','서적문구','사무통신',
            '자동차판매','자동차정비','의료기관','보건위생','요식업소','음료식품','수리서비스']
for i in range(len(cat_code)):
  name_a_to_b(scr,'MCT_CAT_CD',cat_code[i],cat_name[i])

# 시구 및 동 이름 추가
scr = pd.merge(scr, dong_cd, left_on=['GU_CD', 'DONG_CD'], right_on=['GU_CD', 'HDONG_CD'])

# 날짜 형식으로 변환(int->str->date)
scr['STD_YMD'] = pd.to_datetime(scr['STD_DD'].astype(str), format='%Y%m%d',infer_datetime_format=True)

# 클러스터링 추가
scr = pd.merge(scr, dong, left_on='HDONG_NM', right_on='HDONG_NM')

# 연도 추가
scr['STD_Y'] = scr['STD_DD'].astype(str).str.slice(0,4)

scr = scr.drop(['STD_DD', 'GU_CD', 'DONG_CD', 'HDONG_CD', 'AGE_CD'], axis=1)

In [ ]:
scr = scr.groupby(['CLUS_NM', 'HDONG_NM', 'STD_Y', 'STD_YMD', 'MCT_CAT_CD']).agg(sum).reset_index(drop=False)

In [ ]:
scr0 = pd.merge(scr, ska[['HDONG_NM', 'STD_YMD', '유동인구_노원구학원가']], left_on=['HDONG_NM', 'STD_YMD'], right_on=['HDONG_NM', 'STD_YMD'], how='left')
scr1 = pd.merge(scr, ska[['HDONG_NM', 'STD_YMD', '유동인구_노원구대학가']], left_on=['HDONG_NM', 'STD_YMD'], right_on=['HDONG_NM', 'STD_YMD'], how='left')
scr2 = pd.merge(scr, ska[['HDONG_NM', 'STD_YMD', '유동인구_중구소상공']], left_on=['HDONG_NM', 'STD_YMD'], right_on=['HDONG_NM', 'STD_YMD'], how='left')
scr3 = pd.merge(scr, ska[['HDONG_NM', 'STD_YMD', '유동인구_중구빌딩숲']], left_on=['HDONG_NM', 'STD_YMD'], right_on=['HDONG_NM', 'STD_YMD'], how='left')

# **0. 동일하게 적용되는 함수 정의**

In [ ]:
def growth(data):
  sc_cnt_sum_19 =data[data['STD_YMD'] < '2020']
  sc_cnt_sum_20 =data[data['STD_YMD'] > '2020']

  sc_cnt_sum_20.index = sc_cnt_sum_20['STD_YMD']
  del sc_cnt_sum_20['STD_YMD']
  sc_cnt_sum_19.index = sc_cnt_sum_19['STD_YMD']
  del sc_cnt_sum_19['STD_YMD']

  #2020 index 저장
  index_2020 = list(sc_cnt_sum_20.index)

  # rolling 전 요일을 맞춰주기 위한 작업

  #20/02/01 = 일요일
  #19/02/01 = 토요일 ==> 19년 토요일 데이터 삭제
  sc_cnt_sum_19=sc_cnt_sum_19.iloc[1:,]

  #설날 보정
  columns = sc_cnt_sum_19.columns
  index = sc_cnt_sum_19.index

  sc_cnt_sum_19 = np.array(sc_cnt_sum_19)
  sc_cnt_sum_19[2:5,:] = (sc_cnt_sum_19[9:12,:]+sc_cnt_sum_19[16:19,:])/2
  sc_cnt_sum_19 = pd.DataFrame(sc_cnt_sum_19)
  sc_cnt_sum_19.columns = columns
  sc_cnt_sum_19.index = index

  sc_cnt_sum_19_roll7 = pd.DataFrame(sc_cnt_sum_19.rolling(7,min_periods = 1).mean()).reset_index()
  sc_cnt_sum_20_roll7 = pd.DataFrame(sc_cnt_sum_20.rolling(7,min_periods = 1).mean()).reset_index()

  del sc_cnt_sum_19_roll7['STD_YMD']
  del sc_cnt_sum_20_roll7['STD_YMD']

  sc_cnt_sum_20_roll7 = sc_cnt_sum_20_roll7.iloc[:-2,:]

  sc_cnt_sum_per= (sc_cnt_sum_20_roll7-sc_cnt_sum_19_roll7)/sc_cnt_sum_19_roll7

  index_2020=[x.strftime('%m-%d')for x in index_2020]
  sc_cnt_sum_per.index = index_2020[:-2]

  return sc_cnt_sum_per

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

def slope(data):
  cat = data.columns

  r_sq=[]
  slope=[]
  intercept=[]
  for c in cat:
    x = np.arange(0,data.shape[0]/100,0.01).reshape(data.shape[0],-1)
    y = np.array(data.loc[:,c]).reshape(-1,)
    model = LinearRegression().fit(x, y)
    r_sq.append(model.score(x, y))
    intercept.append(model.intercept_)
    slope.append(model.coef_)
  
  return r_sq, intercept, slope

In [ ]:
def draw_slope(data):
  sc_1 =  data.loc[data.index < '02-18',:]
  sc_2 =  data.loc[(data.index >= '02-18') & (data.index < '03-22'),:]
  sc_3 =  data.loc[data.index >= '03-22',:]

  before_r, before_intercept, before_slope = slope(sc_1)
  middle_r, middle_intercept, middle_slope = slope(sc_2)
  after_r, after_intercept, after_slope = slope(sc_3)

  before = pd.DataFrame(
      {
      'slope_before' : np.array(before_slope).reshape(-1,),
      'inter_before' : before_intercept,
      'r2_before': before_r,
  }).T
  before.columns = sc_1.columns

  middle = pd.DataFrame(
      {
      'slope_middle' : - np.array(middle_slope).reshape(-1,),
      'inter_middle' : middle_intercept,
      'r2_middle': middle_r,
  }).T
  middle.columns = sc_2.columns

  after = pd.DataFrame({
      'slope_after': np.array(after_slope).reshape(-1,),
      'inter_after' : after_intercept,
      'r2_after':after_r    
  }).T   
  after.columns = sc_3.columns

  merge = pd.concat([middle.iloc[0,:],after.iloc[0,:]],axis=1)  
  merge['cat'] = merge.index
  merge['cat'] = merge['cat'].apply(lambda x : x[7:])

  return merge

In [ ]:
def draw_slope_tmsc(data, clus):
  sc_1 =  data.loc[data.index < '02-18',:]
  sc_2 =  data.loc[(data.index >= '02-18') & (data.index < '03-22'),:]
  sc_3 =  data.loc[data.index >= '03-22',:]

  before_r, before_intercept, before_slope = slope(sc_1)
  middle_r, middle_intercept, middle_slope = slope(sc_2)
  after_r, after_intercept, after_slope = slope(sc_3)

  before = pd.DataFrame(
      {
      'slope_before' : np.array(before_slope).reshape(-1,),
      'inter_before' : before_intercept,
      'r2_before': before_r,
  }).T
  before.columns = sc_1.columns

  middle = pd.DataFrame(
      {
      'slope_middle' : - np.array(middle_slope).reshape(-1,),
      'inter_middle' : middle_intercept,
      'r2_middle': middle_r,
  }).T
  middle.columns = sc_2.columns

  after = pd.DataFrame({
      'slope_after': np.array(after_slope).reshape(-1,),
      'inter_after' : after_intercept,
      'r2_after':after_r    
  }).T   
  after.columns = sc_3.columns

  merge = pd.concat([middle.iloc[0,:], after.iloc[0,:]], axis=1)  
  merge['cat'] = merge.index
  merge['cat'] = merge['cat'].apply(lambda x : x[7:])
  merge['CLUS_NM'] = clus

  return merge

In [ ]:
def rename_col_tmsc(X):
    X.columns =['SC_CNT_가구', 'SC_CNT_가전',
       'SC_CNT_광학제품', 'SC_CNT_레저업소', 'SC_CNT_레저용품',
       'SC_CNT_문화취미', 'SC_CNT_보건위생', 'SC_CNT_사무통신',
       'SC_CNT_서적문구', 'SC_CNT_수리서비스', 'SC_CNT_숙박',
       'SC_CNT_신변잡화', 'SC_CNT_연료판매', 'SC_CNT_요식업소',
       'SC_CNT_유통업', 'SC_CNT_음료식품', 'SC_CNT_의료기관',
       'SC_CNT_의복', 'SC_CNT_자동차정비', 'SC_CNT_자동차판매',
       'SC_CNT_전기', 'SC_CNT_주방용구', 'SC_CNT_직물',
       'SC_AMT_가구', 'SC_AMT_가전', 'SC_AMT_광학제품',
       'SC_AMT_레저업소', 'SC_AMT_레저용품', 'SC_AMT_문화취미',
       'SC_AMT_보건위생', 'SC_AMT_사무통신', 'SC_AMT_서적문구',
       'SC_AMT_수리서비스', 'SC_AMT_숙박', 'SC_AMT_신변잡화',
       'SC_AMT_연료판매', 'SC_AMT_요식업소', 'SC_AMT_유통업',
       'SC_AMT_음료식품', 'SC_AMT_의료기관', 'SC_AMT_의복',
       'SC_AMT_자동차정비', 'SC_AMT_자동차판매', 'SC_AMT_전기',
       'SC_AMT_주방용구', 'SC_AMT_직물']
    return X

In [ ]:
def add_catclus2(data):
    data['cat_clus'] = 0

    for i in range(len(data)):
        if data.iloc[i,2] in ["직물", "가구", "주방용구", "자동차판매", "레저용품", "자동차판매", "의복", "신변잡화", "가전", "자동차정비"]:
            data.iloc[i,8] = 'CAT1'
        elif data.iloc[i,2] in ["보건위생", "의료기관", "광학제품", "연료판매", "음료식품", "요식업소", "수리서비스"]:
            data.iloc[i,8] = 'CAT2'
        else:
            data.iloc[i,8] = 'CAT3'
    return data

In [ ]:
def remove_cat(data):
    data = data.drop(['SC_CNT_숙박', 'SC_CNT_문화취미', 'SC_CNT_전기', 'SC_CNT_수리서비스'], axis=0)
    return data

In [ ]:
def choose_cat(data, num):
    if num == 1:
        data = data.query('cat in ["직물", "가구", "주방용구", "자동차판매", "레저용품", "자동차판매", "의복", "신변잡화", "가전", "자동차정비"]')
    elif num == 2:
        data = data.query('cat in ["보건위생", "의료기관", "광학제품", "연료판매", "음료식품", "요식업소"]') # 수리서비스
    else:
        data = data.query('cat in ["서적문구", "레저업소", "유통업", "사무통신"]') # 숙박, 전기, 문화취미
    return data

In [ ]:
scr_s3 = scr[scr['CLUS_NM']=='중구 빌딩숲']
scr_s2 = scr[scr['CLUS_NM']=='중구 소상공']
scr_s1 = scr[scr['CLUS_NM']=='노원구 대학가']
scr_s0 = scr[scr['CLUS_NM']=='노원구 학원가']

scr_s3 = scr_s3.groupby(['STD_YMD', 'MCT_CAT_CD']).sum().unstack(level=1).fillna(0)
scr_s2 = scr_s2.groupby(['STD_YMD', 'MCT_CAT_CD']).sum().unstack(level=1).fillna(0)
scr_s1 = scr_s1.groupby(['STD_YMD', 'MCT_CAT_CD']).sum().unstack(level=1).fillna(0)
scr_s0 = scr_s0.groupby(['STD_YMD', 'MCT_CAT_CD']).sum().unstack(level=1).fillna(0)

scr_s3 = rename_col_tmsc(scr_s3)
scr_s2 = rename_col_tmsc(scr_s2)
scr_s1 = rename_col_tmsc(scr_s1)
scr_s0 = rename_col_tmsc(scr_s0)

scr_s3 = scr_s3.iloc[:,:23]
scr_s2 = scr_s2.iloc[:,:23]
scr_s1 = scr_s1.iloc[:,:23]
scr_s0 = scr_s0.iloc[:,:23]

scr_s3 = scr_s3.reset_index(drop=False)
scr_s2 = scr_s2.reset_index(drop=False)
scr_s1 = scr_s1.reset_index(drop=False)
scr_s0 = scr_s0.reset_index(drop=False)

scr_s3 = growth(scr_s3)
scr_s2 = growth(scr_s2)
scr_s1 = growth(scr_s1)
scr_s0 = growth(scr_s0)

# 전기 inf 54개 NA 11개
scr_s3 = scr_s3.fillna(0)

scr_s3 = scr_s3.replace([np.inf, -np.inf], 0)

In [ ]:
# scr = scr.groupby(['CLUS_NM', 'HDONG_NM', 'STD_Y', 'STD_YMD', 'MCT_CAT_CD']).agg(sum).reset_index(drop=False)

# **1. 타격도/회복도 시각화(plane ver.)**

In [ ]:
import plotly.graph_objects as go

def show_scattor_tmsc4(data, x1, x2, y1, y2,name, cat):
    color_discrete_map = {'CAT1': 'rgb(85,107,47)', 'CAT2': 'rgb(169,169,169)', 'CAT3': 'rgb(250,128,114)'}
    fig = px.scatter(data, x="slope_middle", y="slope_after", hover_data=["cat"], text=cat, template='plotly_white', color='cat_clus', color_discrete_map=color_discrete_map) # text="cat", 
    fig.update_traces(textfont_size=15)

    fig.update_traces(textposition='bottom center', 
                      marker=dict(size=15))

    fig.update_layout(
        showlegend=False,
        title=dict(text="회복도 및 타격도 / 클러스터별 / " + name, font=dict(size=20)),
        xaxis_title=dict(text="타격도 (신천지 - 사회적 거리두기)", font=dict(size=16)),
        yaxis_title=dict(text="회복도 (사회적 거리두기 - 최근)", font=dict(size=16)),
        autosize=False,
        width=600,
        height=800,
        margin=dict(l=20,r=20,b=20,t=50,pad=4))

    fig.add_shape(dict(type="line", 
                    x0=0.5, y0=y1,x1=0.5, y1=y2,
                    line=dict(color="red",width=1,dash="dash")))

    fig.add_shape(dict(type="line", 
                    x0=x1, y0=0.5, x1=x2, y1=0.5,
                    line=dict(color="red",width=1,dash="dash")))  
    fig.show()

In [ ]:
def new_cat(data):
     data['catnm1'] = data['cat'].apply(lambda x: '' if x in ['보건위생', '요식업소', '음료식품', '수리서비스', '자동차정비','광학제품', '레저업소', '숙박', '레저용품','의복', '신변잡화'] else x)
     data['catnm2'] = data['cat'].apply(lambda x: '' if x in ['음료식품', '광학제품', '자동차정비'] else x)
     data['catnm3'] = data['cat'].apply(lambda x: '' if x in ['보건위생', '음료식품', '자동차정비','광학제품', '레저업소', '요식업소', '의료기관', '숙박', '레저용품', '신변잡화', '서적문구', '사무통신', '문화취미', '주방용구', '가전', '연료판매'] else x)
     data['catnm4'] = data['cat'].apply(lambda x: '' if x in ['보건위생', '음료식품', '자동차정비', '레저업소', '요식업소', '의료기관', '숙박', '신변잡화', '문화취미', '의복', '수리서비스', '자동차판매', '연료판매'] else x)
     return data

In [ ]:
show_scattor_tmsc4(add_catclus2(new_cat(draw_slope_tmsc(scr_s3, '중구 빌딩숲'))), -2.2, 1.9, -2.5, 2.5,'중구 빌딩숲', 'catnm1')

In [ ]:
show_scattor_tmsc4(add_catclus2(new_cat(draw_slope_tmsc(scr_s2, '중구 소상공'))), -2, 2.21, -0.2, 1.65, '중구 소상공', 'catnm2')

In [ ]:
show_scattor_tmsc4(add_catclus2(new_cat(draw_slope_tmsc(scr_s1, '노원구 대학가'))), -6, 2, -0.5, 4, '노원구 대학가', 'catnm3')

In [ ]:
show_scattor_tmsc4(add_catclus2(new_cat(draw_slope_tmsc(scr_s0, '노원구 학원가'))), -2.5, 3.5, -0.5, 2.7, '노원구 학원가', 'catnm4')

# **2. 타격도/회복도 시각화(유통업 강조 ver.)**

In [ ]:
import plotly.graph_objects as go

def show_scattor_tmsc3(data, x1, x2, y1, y2,name, cat):
    color_discrete_map = {'CAT1': 'rgb(85,107,47)', 'CAT2': 'rgb(169,169,169)', 'CAT3': 'rgb(250,128,114)'}
    fig = px.scatter(data, x="slope_middle", y="slope_after", hover_data=["cat"], template='plotly_white', color='cat_clus', color_discrete_map=color_discrete_map) # text="cat", 
    # fig.update_traces(textfont_size=20)

    fig.update_traces(marker=dict(size=15))

    fig.update_layout(
        showlegend=False,
        title=dict(text="회복도 및 타격도 / 클러스터별 / " + name, font=dict(size=20)),
        xaxis_title=dict(text="타격도 (신천지 - 사회적 거리두기)", font=dict(size=16)),
        yaxis_title=dict(text="회복도 (사회적 거리두기 - 최근)", font=dict(size=16)),
        autosize=False,
        width=600,
        height=700,
        margin=dict(l=20,r=20,b=20,t=50,pad=4),
        shapes=[dict(type="rect",xref="x",yref="y",
                 x0=0.5,y0=0,
                 x1=1.2,y1=0.5,
                 opacity=0.2,fillcolor="pink",line_color="pink")])

    fig.add_shape(dict(type="line", 
                    x0=0.5, y0=y1,x1=0.5, y1=y2,
                    line=dict(color="red",width=1,dash="dash")))

    fig.add_shape(dict(type="line", 
                    x0=x1, y0=0.5, x1=x2, y1=0.5,
                    line=dict(color="red",width=1,dash="dash")))
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=[data.loc['SC_CNT_유통업','slope_middle']],
            y=[data.loc['SC_CNT_유통업','slope_after']],
            marker=dict(
                color='crimson',
                size=25,
                line=dict(
                    color='black',
                    width=5
                )
            ),
            showlegend=False
        )
    )
    fig.show()

In [ ]:
def new_cat2(data):
     data['catnm1'] = data['cat'].apply(lambda x: '' if x not in ['유통업'] else x)
     data['catnm2'] = data['cat'].apply(lambda x: '' if x not in ['유통업'] else x)
     data['catnm3'] = data['cat'].apply(lambda x: '' if x not in ['유통업'] else x)
     data['catnm4'] = data['cat'].apply(lambda x: '' if x not in ['유통업'] else x)
     return data

In [ ]:
show_scattor_tmsc3(add_catclus2(new_cat2(draw_slope_tmsc(scr_s3, '중구 빌딩숲'))), -1.5, 1.5, -2, 2,'중구 빌딩숲', 'catnm1') # x는 (0, 1) y는 (0, 1.2)

In [ ]:
show_scattor_tmsc3(add_catclus2(new_cat2(draw_slope_tmsc(scr_s2, '중구 소상공'))), -1.5, 1.5, -2, 2, '중구 소상공', 'catnm2')

In [ ]:
show_scattor_tmsc3(add_catclus2(new_cat2(draw_slope_tmsc(scr_s1, '노원구 대학가'))),  -1.5, 1.5, -2, 2, '노원구 대학가', 'catnm3')

In [ ]:
show_scattor_tmsc3(add_catclus2(new_cat2(draw_slope_tmsc(scr_s0, '노원구 학원가'))), -1.5, 1.5, -2, 2, '노원구 학원가', 'catnm4')